In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
from pylab import rcParams

import tensorflow as tf
from keras import optimizers, Sequential
from keras.models import Model
from keras.utils import plot_model
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed
from keras.callbacks import ModelCheckpoint, TensorBoard

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score

from numpy.random import seed
seed(7)
import tensorflow as tf
tf.random.set_seed(11) 
from sklearn.model_selection import train_test_split

SEED = 123 #used to help randomly select the data points
DATA_SPLIT_PCT = 0.2

rcParams['figure.figsize'] = 8, 6
LABELS = ["Normal","Break"]

In [ ]:
import pandas as pd
import textblob as tb
from google.colab import files
 
uploaded = files.upload()


Saving data.csv to data.csv


In [ ]:
df = pd.read_csv (r'/content/data.csv',sep=";",quoting=3, on_bad_lines='skip')
print(len(df.index))
print(f"->{df.isin(['NaN', 'None', 'none']).sum()}")
print(df.info())
print(df.head(20))

7331
->text      0
source    0
label     0
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7331 entries, 0 to 7330
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7331 non-null   object
 1   source  7331 non-null   object
 2   label   7331 non-null   object
dtypes: object(3)
memory usage: 171.9+ KB
None
                                                 text     source  label
0   The CDC currently reports 99031 deaths. In gen...   twitter   real 
1    States reported 1121 deaths a small rise from...   twitter   real 
2    Politically Correct Woman (Almost) Uses Pande...  twitter    fake 
3     #IndiaFightsCorona: We have 1524 #COVID test...   twitter   real 
4    Populous states can generate large case count...   twitter   real 
5    "Covid Act Now found ""on average each person...   twitter   real 
6    If you tested positive for #COVID19 and have ...   twitter   real 
7    Obama Calls Trumps Coronavirus

In [ ]:
print(df.duplicated()) #verifico se ci sono duplicati
df.dropna(inplace = True) #rimuovo le righe con valore NULL
df.drop_duplicates(inplace = True)#elimino duplicati
print(df.info())
print(df.head())

0       False
1       False
2       False
3       False
4       False
        ...  
7326    False
7327    False
7328    False
7329    False
7330    False
Length: 7331, dtype: bool
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7276 entries, 0 to 7330
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    7276 non-null   object
 1   source  7276 non-null   object
 2   label   7276 non-null   object
dtypes: object(3)
memory usage: 227.4+ KB
None
                                                text     source  label
0  The CDC currently reports 99031 deaths. In gen...   twitter   real 
1   States reported 1121 deaths a small rise from...   twitter   real 
2   Politically Correct Woman (Almost) Uses Pande...  twitter    fake 
3    #IndiaFightsCorona: We have 1524 #COVID test...   twitter   real 
4   Populous states can generate large case count...   twitter   real 


In [ ]:
#rimuovo la punteggiatura tranne ! ?
print(df.head(20))
df = df.apply(lambda x: x.str.replace('\d+', ''))
#df['Name'] = df['Name'].str.replace('\d+', '')
df = df.apply(lambda x: x.str.replace('@', ''))
df = df.apply(lambda x: x.str.replace('?', ''))
df = df.apply(lambda x: x.str.replace('!', ''))
df = df.apply(lambda x: x.str.replace('&', ''))
df = df.apply(lambda x: x.str.replace(',', ''))
df = df.apply(lambda x: x.str.replace(':', ''))
df = df.apply(lambda x: x.str.replace('.', ''))
df = df.apply(lambda x: x.str.replace('%', ''))
df = df.apply(lambda x: x.str.replace('"', ''))
df = df.apply(lambda x: x.str.replace('-', ''))
df = df.apply(lambda x: x.str.replace('#', ''))
df = df.apply(lambda x: x.str.replace("'", ''))
df.replace('\'','', regex=True, inplace=True) 
df = df.apply(lambda x: x.str.replace('(', ''))
df = df.apply(lambda x: x.str.replace(')', ''))
print(df.head(20))

                                                 text     source  label
0   The CDC currently reports 99031 deaths. In gen...   twitter   real 
1    States reported 1121 deaths a small rise from...   twitter   real 
2    Politically Correct Woman (Almost) Uses Pande...  twitter    fake 
3     #IndiaFightsCorona: We have 1524 #COVID test...   twitter   real 
4    Populous states can generate large case count...   twitter   real 
5    "Covid Act Now found ""on average each person...   twitter   real 
6    If you tested positive for #COVID19 and have ...   twitter   real 
7    Obama Calls Trumps Coronavirus Response A Cha...  twitter    fake 
8     ???Clearly, the Obama administration did not...  twitter    fake 
9     RetractionHydroxychloroquine or chloroquine ...   twitter   fake 
10   Take simple daily precautions to help prevent...   twitter   real 
11   The NBA is poised to restart this month. In M...   twitter   fake 
12   We just announced that the first participants...   twitter 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  # This is added back by InteractiveShellApp.init_path()


                                                 text     source  label
0   The CDC currently reports  deaths In general t...   twitter   real 
1    States reported  deaths a small rise from las...   twitter   real 
2    Politically Correct Woman Almost Uses Pandemi...  twitter    fake 
3     IndiaFightsCorona We have  COVID testing lab...   twitter   real 
4    Populous states can generate large case count...   twitter   real 
5    Covid Act Now found on average each person in...   twitter   real 
6    If you tested positive for COVID and have no ...   twitter   real 
7    Obama Calls Trumps Coronavirus Response A Cha...  twitter    fake 
8     Clearly the Obama administration did not lea...  twitter    fake 
9     RetractionHydroxychloroquine or chloroquine ...   twitter   fake 
10   Take simple daily precautions to help prevent...   twitter   real 
11   The NBA is poised to restart this month In Ma...   twitter   fake 
12   We just announced that the first participants...   twitter 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [ ]:
from textblob import TextBlob
from textblob.en.inflect import singularize
from textblob import Word

df['text'] = df['text'].astype('str')
print(df.head(20))
#rendo singolari le parole
df['text'] = df['text'].apply(lambda x: ' '.join([singularize(item) for item in x.split()]))
print(df.head(20))
#df['text'] = df['text'].apply(lambda x: ' '.join([lemmatize(item) for item in x.split()]))
#print(df.head(20))

                                                 text     source  label
0   The CDC currently reports  deaths In general t...   twitter   real 
1    States reported  deaths a small rise from las...   twitter   real 
2    Politically Correct Woman Almost Uses Pandemi...  twitter    fake 
3     IndiaFightsCorona We have  COVID testing lab...   twitter   real 
4    Populous states can generate large case count...   twitter   real 
5    Covid Act Now found on average each person in...   twitter   real 
6    If you tested positive for COVID and have no ...   twitter   real 
7    Obama Calls Trumps Coronavirus Response A Cha...  twitter    fake 
8     Clearly the Obama administration did not lea...  twitter    fake 
9     RetractionHydroxychloroquine or chloroquine ...   twitter   fake 
10   Take simple daily precautions to help prevent...   twitter   real 
11   The NBA is poised to restart this month In Ma...   twitter   fake 
12   We just announced that the first participants...   twitter 

In [ ]:
#La lemmatizzazione di textBlob richiede di indicare se deve lemmatizzare un verbo oun sostantivo. dato che non ho previsto una fase di POS, ho deciso di lemmatizzare soltanto i verbi in quanto sembrano essere quelli che hanno più necessità e che racchiudono il significato maggiore.
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from textblob import Word
print(df.head(20))
df['text'] = df['text'].astype('str')
df['text'] = df['text'].apply(lambda x: ' '.join([Word(item).lemmatize("v") for item in x.split()]))
print(df.head(20))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


                                                 text     source  label
0   The CDC currently report death In general the ...   twitter   real 
1   State reported death a small rise from last Tu...   twitter   real 
2   Politically Correct Woman Almost Use Pandemic ...  twitter    fake 
3   IndiaFightsCorona We have COVID testing labora...   twitter   real 
4   Populou state can generate large case count bu...   twitter   real 
5   Covid Act Now found on average each person in ...   twitter   real 
6   If you tested positive for COVID and have no s...   twitter   real 
7   Obama Call Trump Coronaviru Response A Chaotic...  twitter    fake 
8   Clearly the Obama administration did not leave...  twitter    fake 
9   RetractionHydroxychloroquine or chloroquine wi...   twitter   fake 
10  Take simple daily precaution to help prevent t...   twitter   real 
11  The NBA is poised to restart thi month In Marc...   twitter   fake 
12  We just announced that the first participant i...   twitter 

In [ ]:
print(df.head(20))
stopWords = {"the": "",
            "The": "",
            "A": "",
            "a": "",
            "an": "",
            "An": "",
            "I": "",
            "You": "",
            "you": "",
            "It": "",
            "it": "",
            "he": "",
            "He": "",
            "She": "",
            "she": "",
            "we": "",
            "We": "",
            "They": "",
            "they": "",
            "me": "",
            "Me": "",
            "him": "",
            "Him": "",
            "her": "",
            "Her": "",
            "us": "",
            "Us": "",
            "them": "",
            "Them": "",
            "By": "",
            "by": "",
            "within": "",
            "Within": "",
            "From": "",
            "from": "",
            "To": "",
            "to": "",
            "After": "",
            "after": "",
            "util": "",
            "Until": "",
            "Til": "",
            "til": "",
            "before": "",	
            "Before": "",
            "since": "",
            "Since": "",
            "for": "",
            "For": "",	
            "During": "",
            "at": "",
            "At": "",
            "In": "",
            "in": "",
            "On": "",
            "on": "",
            "Over": "",
            "over": "",
            "Above": "",
            "above": "",
            "Under": "",
            "under": "",
            "Around": "",
            "around": "",
            "Near": "",
            "near": "",
            "Behind": "",
            "behind": "",
            "Across": "",
            "across": "",
            "between": "",
            "Between": "",
            "of": "",
            "Of": "",
            "Among": "",
            "among": "",
            "Into": "",
            "into": "",
            "by": "",
            "By": "",
            "As": "",
            "as": "",
            "And": "",
            "and": "",
            "End": "",
            "end": ""}
print(stopWords.get("As"))
df['text'] = df['text'].astype('str')
column_names = ["text", "source", "lable"]
df1 = pd.DataFrame(columns = column_names)
c=0
for x in df['text']:
  var=x.split(" ")
  i=0
  for v in var:
    if (stopWords.get(v) == ""):
      var[i] = stopWords.get(v)
    i+=1
  if c == 1:
    print(var) 
  
  df.iloc[c] = pd.Series([' '.join(var),df['source'].iloc[c],df['label'].iloc[c]])  
    
  c+=1
      
print(var)
print(df.head())

                                                 text     source  label
0   The CDC currently report death In general the ...   twitter   real 
1   State report death a small rise from last Tues...   twitter   real 
2   Politically Correct Woman Almost Use Pandemic ...  twitter    fake 
3   IndiaFightsCorona We have COVID test laborator...   twitter   real 
4   Populou state can generate large case count bu...   twitter   real 
5   Covid Act Now find on average each person in I...   twitter   real 
6   If you test positive for COVID and have no sym...   twitter   real 
7   Obama Call Trump Coronaviru Response A Chaotic...  twitter    fake 
8   Clearly the Obama administration do not leave ...  twitter    fake 
9   RetractionHydroxychloroquine or chloroquine wi...   twitter   fake 
10  Take simple daily precaution to help prevent t...   twitter   real 
11  The NBA be poise to restart thi month In March...   twitter   fake 
12  We just announce that the first participant in...   twitter 

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
# "min_df" o la frequenza minima del documento. Imposta il numero minimo di documenti in cui è contenuto qualsiasi termine. 
#Può essere un numero intero che imposta il numero in modo specifico o un decimale compreso tra 0 e 1 che viene interpretato come percentuale di tutti i documenti. 
#Il prossimo è "max_df" che controlla in modo simile il numero massimo di documenti in cui è possibile trovare qualsiasi termine.
#Se il 90% dei documenti contiene la parola "spork", è così comune che non è molto utile. Infine, abbiamo "ngram_range" che è una tupla contenente l'intervallo di dimensioni di ngram da utilizzare. 
#(1, 3) significa utilizzare 1 grammo, 2 grammi e 3 grammi. Stiamo solo usando 1 e 2 grammi.
cvec = CountVectorizer(min_df=.0025, max_df=.1, ngram_range=(1,2))

#Calcola tutti gli n-grammi trovati in tutti i documenti
from itertools import islice
cvec.fit(df.text)
list(islice(cvec.vocabulary_.items(), 20))

#Controlla quanti n-grammi totali abbiamo
len(cvec.vocabulary_)

#trasformare il documento in una rappresentazione di "Bag of words" che essenzialmente è solo una colonna separata per ogni termine contenente il conteggio all'interno di ogni documento.
#Successivamente, daremo un'occhiata alla scarsità di questa rappresentazione che ci consente di sapere quanti valori diversi da zero ci sono nel set di dati. 
#Più i dati sono scarsi, più difficile sarà modellare
cvec_counts = cvec.transform(df.text)
print('sparse matrix shape:', cvec_counts.shape)
print('conteggio diverso da zero:', cvec_counts.nnz)
print('scarsità: %.2f%%' % (100.0 * cvec_counts.nnz / (cvec_counts.shape[0] * cvec_counts.shape[1])))

#i primi 20 termini più comuni
import numpy as np
occ = np.asarray(cvec_counts.sum(axis=0)).ravel().tolist()
counts_df = pd.DataFrame({'term': cvec.get_feature_names(), 'occurrences': occ})
counts_df.sort_values(by='occurrences', ascending=False).head(20)

#Ora che abbiamo il conteggio dei termini per ogni documento, possiamo usare TfidfTransformer per calcolare i pesi per ogni termine in ogni documento
transformer = TfidfTransformer()
transformed_weights = transformer.fit_transform(cvec_counts)
print(transformed_weights.shape)

#i primi 20 termini in base al peso medio di tf-idf:
weights = np.asarray(transformed_weights.mean(axis=0)).ravel().tolist()
weights_df = pd.DataFrame({'term': cvec.get_feature_names(), 'weight': weights})
weights_df.sort_values(by='weight', ascending=False).head(20)


sparse matrix shape: (7276, 3409)
conteggio diverso da zero: 230717
scarsità: 0.93%
(7276, 3409)


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,term,weight
3136,vaccine,0.013621
856,day,0.012800
3374,ymy,0.012387
2156,patient,0.011252
2883,than,0.011175
2134,pandemic,0.011133
1485,if,0.010756
3029,total,0.010651
673,confirm,0.010224
1444,hospital,0.010175


In [ ]:
df["label"].loc[df["label"]=="real "]=1.0
df["label"].loc[df["label"]=="fake "]=0.0
print(df.head())

                                                text     source label
0   CDC currently report death  general  discrepa...   twitter    1.0
1  State report death  small rise  last Tuesday S...   twitter    1.0
2  Politically Correct Woman Almost Use Pandemic ...  twitter     0.0
3  IndiaFightsCorona  have COVID test laboratory ...   twitter    1.0
4  Populou state can generate large case count bu...   twitter    1.0


In [ ]:
def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Raccoglie i record passati fino al periodo di ricerca
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
    return output_X, output_y

In [ ]:
# definisce le serie temporali di input serie
timeseries =  transformed_weights.transpose()

timesteps = timeseries.shape[0]
n_features = timeseries.shape[1]

In [ ]:
timesteps = 1
X, y = temporalize(X = transformed_weights.toarray(), y = np.zeros(len(transformed_weights.toarray())), lookback = timesteps)
#X, y = temporalize(X = transformed_weights.toarray(), y = np.zeros(cvec_counts.shape[0]), lookback = timesteps)

n_features = transformed_weights.shape[1]
X = np.array(X)
X = X.reshape(X.shape[0], timesteps, n_features)

In [ ]:
# define model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_8 (LSTM)               (None, 1, 128)            1811456   
                                                                 
 lstm_9 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector_2 (RepeatVect  (None, 1, 64)            0         
 or)                                                             
                                                                 
 lstm_10 (LSTM)              (None, 1, 64)             33024     
                                                                 
 lstm_11 (LSTM)              (None, 1, 128)            98816     
                                                                 
 time_distributed_2 (TimeDis  (None, 1, 3409)          439761    
 tributed)                                            

In [ ]:
from sklearn.model_selection import train_test_split
X_train , X_test , y_train , y_test = train_test_split ( X , y , test_size = 0.2 , random_state = 1 )

In [ ]:
model.fit(X, X, epochs=5, batch_size=5, verbose=0)

In [ ]:

# demonstrate reconstruction
y_pred = model.predict(X_test, verbose=0)
print('---Predicted---')
print(np.round(y_pred,3))
print('---Actual---')
print(np.round(X, 3))

---Predicted---
[[[ 0.005  0.002  0.    ... -0.     0.    -0.   ]]

 [[ 0.005  0.002  0.    ... -0.     0.    -0.   ]]

 [[ 0.005  0.002  0.    ... -0.     0.    -0.   ]]

 ...

 [[ 0.005  0.002  0.    ... -0.     0.    -0.   ]]

 [[ 0.005  0.002  0.    ... -0.     0.    -0.   ]]

 [[ 0.005  0.002  0.    ... -0.     0.    -0.   ]]]
---Actual---
[[[0.    0.    0.    ... 0.    0.    0.   ]]

 [[0.    0.    0.    ... 0.    0.    0.   ]]

 [[0.    0.    0.    ... 0.    0.    0.   ]]

 ...

 [[0.    0.039 0.    ... 0.    0.    0.   ]]

 [[0.    0.    0.    ... 0.    0.    0.   ]]

 [[0.    0.    0.    ... 0.    0.    0.   ]]]
